# Game Mining Demo

In [1]:
import pandas as pd
import sys
import numpy as np
import os
import plotly.express as px


sys.path.append('../')

In [2]:
DATA_PATH = '../data/simulation_output/'
files = sorted(f for f in os.listdir(DATA_PATH) if 'pkl.gz' in f)
last_file = files[-1]
FILE_PATH = DATA_PATH + last_file
print(FILE_PATH)

../data/simulation_output/2022-01-24_10-46-34.pkl.gz


In [3]:
sim_df = pd.read_pickle(FILE_PATH).assign(run=lambda df: df.run.astype(str))
sim_df.head(2)

,players,actions,past_actions,payoffs,payoff_tensor,simulation,subset,run,timestep,payoff_shift_interval,payoff_intensity
0,"[Player(player_no=0, likelihood_vector={0: 0.5...",{},"{0: [], 1: []}",{},"[[[1, 0.5], [0.5, 0.1]], [[1, 0.5], [0.5, 0.1]]]",0,0,1,0,10,0.1
2,"[Player(player_no=0, likelihood_vector=(0.5, 0...","{0: 1, 1: 1}","{0: [1], 1: [1]}","{0: 0.1, 1: 0.1}","[[[1.0, 0.5], [0.5, 0.1]], [[1.0, 0.5], [0.5, ...",0,0,1,1,10,0.1


In [4]:
COLS_TO_DROP = ['actions', 'payoffs', 'players', 'past_actions',
                'payoff_tensor', 'subset']
INDICES = ['payoff_shift_interval', 'payoff_intensity', 'run', 'timestep']


df = (sim_df.assign(player_1_action=lambda df: df.actions.map(lambda x: x.get(0, None)))
            .assign(player_2_action=lambda df: df.actions.map(lambda x: x.get(1, None)))
            .assign(player_1_likelihood=lambda df: df.players.map(lambda x: x[1].likelihood_vector[0]))
            .assign(player_2_likelihood=lambda df: df.players.map(lambda x: x[0].likelihood_vector[0]))
            .assign(player_1_payoff=lambda df: df.payoffs.map(lambda x: x.get(0, None)))
            .assign(player_2_payoff=lambda df: df.payoffs.map(lambda x: x.get(1, None)))
            .assign(player_2_payoff=lambda df: df.payoffs.map(lambda x: x.get(1, None)))
            .assign(payoff_tensor_mean=lambda df: df.payoff_tensor.map(np.array).map(np.mean))
            .assign(payoff_tensor_shift=lambda df: df.payoff_tensor_mean.diff() != 0.0)
            .drop(columns=COLS_TO_DROP)
            .dropna()
            .set_index(INDICES)

    )

df.sample(4)

simulation  \
payoff_shift_interval payoff_intensity run timestep               
100                   2.0              2   435                0   
10                    0.1              5   475                0   
                      2.0              4   235                0   
                                           300                0   

                                                     player_1_action  \
payoff_shift_interval payoff_intensity run timestep                    
100                   2.0              2   435                   0.0   
10                    0.1              5   475                   0.0   
                      2.0              4   235                   1.0   
                                           300                   1.0   

                                                     player_2_action  \
payoff_shift_interval payoff_intensity run timestep                    
100                   2.0              2   435                   1.0   
10                    0.1              5   475                   0.0   
                      2.0              4   235                   1.0   
                                           300                   1.0   

                                                     player_1_likelihood  \
payoff_shift_interval payoff_intensity run timestep                        
100                   2.0              2   435                  0.046083   
10                    0.1              5   475                  0.413924   
                      2.0              4   235                  0.540171   
                                           300                  0.553177   

                                                     player_2_likelihood  \
payoff_shift_interval payoff_intensity run timestep                        
100                   2.0              2   435                  0.461751   
10                    0.1              5   475                  0.124473   
                      2.0              4   235                  0.588889   
                                           300                  0.591304   

                                                     player_1_payoff  \
payoff_shift_interval payoff_intensity run timestep                    
100                   2.0              2   435             -3.443403   
10                    0.1              5   475              0.226503   
                      2.0              4   235            -11.984994   
                                           300             -9.316501   

                                                     player_2_payoff  \
payoff_shift_interval payoff_intensity run timestep                    
100                   2.0              2   435              5.812765   
10                    0.1              5   475              1.069122   
                      2.0              4   235             -4.042236   
                                           300             -9.934215   

                                                     payoff_tensor_mean  \
payoff_shift_interval payoff_intensity run timestep                       
100                   2.0              2   435                -0.599558   
10                    0.1              5   475                 0.255189   
                      2.0              4   235                -2.956952   
                                           300                -3.319713   

                                                     payoff_tensor_shift  
payoff_shift_interval payoff_intensity run timestep                       
100                   2.0              2   435                     False  
10                    0.1              5   475                     False  
                      2.0              4   235                     False  
                                           300                      True

In [5]:
fig_df = df.reset_index()

fig = px.line(fig_df,
              x='timestep',
              y='payoff_tensor_mean',
              color='run',
              facet_col='payoff_shift_interval',
              facet_row='payoff_intensity')
fig.show()


In [6]:
fig_df = df.reset_index()

fig = px.line(fig_df,
              x='timestep',
              y='player_1_likelihood',
              color='run',
              facet_col='payoff_shift_interval',
              facet_row='payoff_intensity')
fig.show()


In [7]:
fig_df = df.reset_index()

fig = px.density_heatmap(fig_df,
              x='timestep',
              y='player_1_action',
              facet_col='payoff_shift_interval',
              facet_row='payoff_intensity')
fig.show()


In [8]:
ID_VARS = ['timestep', 'run', 'payoff_shift_interval', 'payoff_intensity']
VARS = ['player_1_action', 'player_2_action']
fig_df = df.reset_index().melt(id_vars=ID_VARS, value_vars=VARS)

fig = px.density_heatmap(fig_df,
              x='timestep',
              y='value',
              facet_row='variable',
              nbinsx=100)
fig.show()

In [9]:
ID_VARS = ['timestep', 'run', 'payoff_shift_interval', 'payoff_intensity']
VARS = ['player_1_likelihood', 'player_2_likelihood']
fig_df = df.reset_index().melt(id_vars=ID_VARS, value_vars=VARS)

fig = px.scatter(fig_df,
              x='timestep',
              y='value',
              color='run',
              facet_row='variable',
              marginal_y='box',
              #nbinsx=100
              )
fig.show()